# Effects of aging on Drosophila germ granule morphology and aggregation #

### Table of Contents
1. [Using cluster analysis to identify individual germ granules in aged and wild type Drosophila germ plasm](#1.-Using-cluster-analysis-to-identify-individual-germ-granules-in-aged-and-wild-type-Drosophila-germ-plasm)
 1. [Data acquisition and feature extraction](#A.-Data-acquisition-and-feature-extraction)
 1. [Introduction to cluster analysis and suitability of density based clustering to identify individual granules in the 3D point cloud](#B.-Introduction-to-cluster-analysis-and-suitability-of-density-based-clustering-to-identify-individual-granules-in-the-3D-point-cloud)
 1. [Discussion of density based clustering algorithms](#C.-Discussion-of-density-based-clustering-algorithms)
 1. [Comparison of density based clustering algorithms and parameter selection using the Silhouette coefficient](#D.-Comparison-of-density-based-clustering-algorithms-and-parameter-selection-using-the-Silhouette-coefficient)
 1. [Preliminary Results](#E.-Preliminary-Results)
 1. [Directions](#F.-Directions)

## 1. Using cluster analysis to identify individual germ granules in aged and wild type Drosophila germ plasm ##

### A. Data acquisition and feature extraction ###
Structured illumination microscopy was used to obtain z-stacks of vasaGFP and oskGFP polar granules in the germ plasm of Drosophila embryos. Each z-slice is a 16 bit grayscale image. The z-stack is like a three dimensional matrix of scalar intensity values ranging from 0-65535. Using the gray level stack histogram, Yen thresholding was applied to the z-stack to eliminate background fluorescence. The thresholded stack is like a binary three dimensional matrix in which entries of zero represent background and 1(or 65535) represents foreground. The location of high fluorescent intensity regions, i.e. foreground, was extracted from the sparse binary matrix. Row number, column number and z-slice number of high fluorescent intensity regions,  scaled to reflect biological size, was used as x, y, and z dimensional data respectively. In this way the binary sparse matrix is converted to a 3D point cloud. The conversion of the z-stack to a 3D point cloud is convenient for further computational analysis as it removes the need to manipulate large three dimensional matrices and allows us to consider only regions of fluorescence within the matrix. 



### B. Introduction to cluster analysis and suitability of density based clustering to identify individual granules in the 3D point cloud ###
While the goal of supervised machine learning is to learn the function that was used to assign labels to a dataset so that new data can be categorized accurately, unsupervised machine learning tries to assign labels to data given no a-priori knowledge about the categories to which they belong. Clustering is an unsupervised machine learning task, which can be formulated as an optimization problem. Variability of a cluster, V, and dissimilarity  of the set of clusters obtained, D, are the metrics of the optimization problem.


$$\begin{aligned}
\text{Let n be the number of data points.} \\
\text{Let c represent a single cluster.} \\
\text{Let C represent a set of clusters.} \\
\text{The variability of a cluster:} \\
V(c) \ &= \sum_{e\in c} distance(mean(c),e)^2 \\
\text{where e is every data point in the cluster.} \\
\text{The dissimilarity of a set of clusters:} \\
D(C) \ &= \sum_{c\in C} V(c) \\
\text{We want to find a set of clusters such that:} \\
  {\text{minimize}}\qquad& D(C) \\
    \text{subject to:}\qquad& the\ number\ of\ clusters\ \le n \\
    \end{aligned}$$
(Algorithm adapted from [MIT OCW 6.0002 Lecture 12](https://www.youtube.com/watch?v=esmzYhuFnds&t=475s&frags=pl%2Cwn.pdf) )

KMeans and Hierarchical Agglomerative Clustering are two common clustering algorithms based on these metrics. The drawbacks of these algorithms are that they can only be applied when we expect clusters to be globular and we know the number of clusters beforehand. Density based clustering algorithms, designed to cluster regions of high density separated by regions of low density, are well suited for clustering point clouds with no prior knowledge of the number or  shape of clusters. Density based clustering is best suited to identify individual germ granules in wild type and aged Drosophila embryos since we have little a-priori knowledge about the number, size and morphology of these granules. 

### C. Discussion of density based clustering algorithms ###

#### a. DBSCAN (Density based spatial clustering of applications with noise)  ####


Quick links: 
([DBSCAN wiki](https://en.wikipedia.org/wiki/DBSCAN))
([Original paper](https://www.aaai.org/Papers/KDD/1996/KDD96-037.pdf)) 
([Visualize DBSCAN in action](https://www.naftaliharris.com/blog/visualizing-dbscan-clustering/))

#### c. HDBSCAN (Hierarchical density based spatial clustering of applications with noise)  ####

Quick links: 
([How HDBSCAN works](https://hdbscan.readthedocs.io/en/latest/how_hdbscan_works.html)) 
([Original paper](https://link.springer.com/chapter/10.1007/978-3-642-37456-2_14))

Notes on the process:
Overcomes the drawbacks of DBSCAN by converting it into a hierarchical clustering algorithm, and then extracts flat clusters from the hierarchy. 

Steps of HDBSCAN: 
- Transform the space according to the density/sparsity.
- Build the minimum spanning tree of the distance weighted graph.
- Construct a cluster hierarchy of connected components.
- Condense the cluster hierarchy based on minimum cluster size.
- Extract the stable clusters from the condensed tree.

Transform the space based on density 
Assign values to points depending on the density of points around it.
An inexpensive estimate of density is the kth nearest neighbor.
We define core distance for a parameter k for every point in the space as:
Define a new distance metric called mutual reachability distance:

Core distance - defined for every point 
Mutual reachability distance - defined between pairs of points 

Dense points remain the same distance from each other but sparser points are pushed futher away in this transformed space. 

better allows us to estimate the hierarchy of the true density distribution from which the points were drawn under this transformed space. 

A fully connected graph based on mutual reachability distances is constructed 

Obtain the MST of the compete graph
what is a MST? - A MST is a subset of the edges of a connected graph that connects all the vertices together, without any cycles, with the minimum total weight possible. 

It is a way of reducing the computational complexity but ensuring all points are connected. 
It eases the process of building a hierarchy becuase if one edge is removed, two connected componenets are obtained.


Consider having a threshold value that is initially high and is lowered
as we drop edges we will start to disconnect the graph into connected componenets - and evetually we will have a hierarchy of connected components at varying threshold levels/ corresponding to various epsilons 

Getting a set of flat clusters from the hierarchy
DBSCAN draws a line through for a constant epsilon and takes those clusters

We want to do better
First prune the tree - we need to define minimum cluster size 
prune off the clusters that are smaller than minimum clsuter size 
think of points falling out of a cluster as we decrease epsilon 
unless the points falling off are big enough to be considered as a separate cluster, we keep it with the same cluster.

width of the line represents the number of points in the cluster 
each node has data about how the size of the cluster decreases over varying distance 

min cluster size = 5

we then chose the number of clustes that persist and have a longer lifetime 
and if you select a cluster, you can't select any of its descendants 


Silhouette
it is a measure of how well each data point has been clusterd, or a meausre 





### D. Comparison of density based clustering algorithms and parameter selection using the Silhouette coefficient  ###

### E. Preliminary Results ###

### F. Directions ###